In [1]:
import pandas as pd
import time
import numpy as np
import os
import pandas_gbq
from google.cloud import bigquery
import basedosdados as bd

In [2]:
query = "SELECT * FROM `basedosdados.br_cgu_ebt.municipio` "

In [3]:
client = bigquery.Client()


In [4]:
df = pandas_gbq.read_gbq(query, project_id='repositoriodedadosgpsp')

Downloading: 100%|██████████| 1330/1330 [00:00<00:00, 3344.92rows/s]


In [5]:
df

,ano,sigla_uf,id_municipio,ranking,nota
0,2020,ES,3205002,1,9.95
1,2020,PR,4113700,1,9.95
2,2020,PA,1505304,3,9.92
3,2020,PE,2611606,4,9.90
4,2020,ES,3205309,5,9.85
...,...,...,...,...,...
1325,2021,GO,5219753,558,5.01
1326,2021,GO,5220454,133,8.90
1327,2021,GO,5221403,49,9.54
1328,2021,GO,5221858,258,7.50


In [6]:
## Chamar todos os municípios para das Nan quando for 0. 

os.chdir("G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS\\415 - Repositório de Dados\Repositório Local\\IBGE")

In [30]:
mun = pd.read_csv('Censo_previa_pop.csv',sep=";")

In [35]:
mun=mun[['id_municipio']]

In [36]:
df['id_municipio']= pd.to_numeric(df['id_municipio'])

In [25]:
df.columns

Index(['ano', 'sigla_uf', 'id_municipio', 'ranking', 'nota'], dtype='object')

In [37]:
mun['ano']=np.nan
mun['sigla_uf']=np.nan
mun['ranking']=np.nan
mun['nota']=np.nan

## tem que fazer os tipos serem os mesmos também.
#Aqui eu adiciono as mesmas colunas para que seja feito o merge



In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1330 entries, 0 to 1329
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ano           1330 non-null   Int64  
 1   sigla_uf      1330 non-null   object 
 2   id_municipio  1330 non-null   int64  
 3   ranking       1330 non-null   Int64  
 4   nota          1330 non-null   float64
dtypes: Int64(2), float64(1), int64(1), object(1)
memory usage: 54.7+ KB


In [41]:
mun.columns

Index(['id_municipio', 'ano', 'sigla_uf', 'ranking', 'nota'], dtype='object')

In [42]:
mun=mun[['ano', 'sigla_uf','id_municipio',  'ranking', 'nota']]

In [63]:
mun.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ano           0 non-null      Int64  
 1   sigla_uf      0 non-null      object 
 2   id_municipio  5570 non-null   int64  
 3   ranking       0 non-null      Int64  
 4   nota          0 non-null      float64
dtypes: Int64(2), float64(1), int64(1), object(1)
memory usage: 228.6+ KB


In [74]:
df1=pd.concat([df,mun],ignore_index=True)

In [77]:
client = bigquery.Client()
dataset_ref = client.dataset('participacao_transparencia')

In [75]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6900 entries, 0 to 6899
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ano           1330 non-null   Int64  
 1   sigla_uf      1330 non-null   object 
 2   id_municipio  6900 non-null   int64  
 3   ranking       1330 non-null   Int64  
 4   nota          1330 non-null   float64
dtypes: Int64(2), float64(1), int64(1), object(1)
memory usage: 283.1+ KB


In [80]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de referencia da informacao'),
 bigquery.SchemaField('sigla_uf','STRING',description='Sigla da Unidade da Federação.'),
 bigquery.SchemaField('id_municipio','FLOAT',description='Identificador do município pelo IBGE'),
 bigquery.SchemaField('ranking','INTEGER',description='ranking dos municípios'),
 bigquery.SchemaField('nota','Float',description='nota daquele município')
 ]

In [81]:
table_ref = dataset_ref.table('CGU_ranking_transparencia_municipio') # nome da tabela no padrão FONTE_algo_intuitivo_dado
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df1, table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=4f3a31e9-c4cc-40e5-8889-99a2fdbd88fb>